In [ ]:
import copy
import cv2
import datetime
import glob
import h5py
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
import scipy.signal
import skimage
import sys
import tifffile
import time
from pathlib import Path
from lbm_util import init_params

plt.rcParams["figure.dpi"] = 900

try:
    from icecream import ic, install, argumentToString

    install()
except ImportError:  # graceful fallback if icecream isn't installed.
    ic = lambda *a: None if not a else (a[0] if len(a) == 1 else a)  # noqa


@argumentToString.register(np.ndarray)
def _(obj):
    """Format our return for the icecream debug statement."""
    return f"ndarray, shape={obj.shape}, dtype={obj.dtype}"


params = init_params()

ic.enable()

tiff_path = Path(/v-data4/foconnell/data/lbm)

if params['debug']:
    # %% Get MROI info from tif metadata
    with tifffile.TiffFile(path_input_file) as tif:
        metadata = {}
        for tag in tif.pages[0].tags.values():
            tag_name, tag_value = tag.name, tag.value
            metadata[tag_name] = tag_value

    mrois_si_raw = json.loads(metadata["Artist"])["RoiGroups"]["imagingRoiGroup"]["rois"]
    if type(mrois_si_raw) != dict:
        mrois_si = []
        for roi in mrois_si_raw:
            if type(roi["scanfields"]) != list:  # TODO: eval
                scanfield = roi["scanfields"]
            else:
                scanfield = roi["scanfields"][
                    np.where(np.array(roi["zs"]) == 0)[0][0]
                ]
            roi_dict = {}
            roi_dict["center"] = np.array(scanfield["centerXY"])
            roi_dict["sizeXY"] = np.array(scanfield["sizeXY"])
            roi_dict["pixXY"] = np.array(scanfield["pixelResolutionXY"])
            mrois_si.append(roi_dict)
    else:
        scanfield = mrois_si_raw["scanfields"]
        roi_dict = {}
        roi_dict["center"] = np.array(scanfield["centerXY"])
        roi_dict["sizeXY"] = np.array(scanfield["sizeXY"])
        roi_dict["pixXY"] = np.array(scanfield["pixelResolutionXY"])
        mrois_si = [roi_dict]

    # Sort MROIs so they go from left-to-right (but keep the un-sorted because that matches how they were acquired and saved in the long-tif-strip)
    mrois_centers_si = np.array([mroi_si["center"] for mroi_si in mrois_si])
    x_sorted = np.argsort(mrois_centers_si[:, 0])
    mrois_si_sorted_x = [mrois_si[i] for i in x_sorted]
    mrois_centers_si_sorted_x = [mrois_centers_si[i] for i in x_sorted]
    # ----
    # %% Load, reshape (so time and planes are 2 independent dimensions) and re-order (planes, fix Jeff's order)
ic("Loading file (expect warning for multi-file recording)")

tiff_file = tifffile.imread(path_input_file)
dim1 = tiff_file.shape[0]
dim2 = tiff_file.shape[1]
dim3 = tiff_file.shape[2]
